In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
import numpy as np
import logging

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='feature_engineering_with_analysis.log',
    filemode='w'
)
logger = logging.getLogger("FeatureEngineeringLogger")
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_format = logging.Formatter('%(levelname)s - %(message)s')
console_handler.setFormatter(console_format)
logger.addHandler(console_handler)

# Datos simulados con Gasto_Anual como variable objetivo
data = {
    'Edad': [25, 30, 45, 35, 50],
    'Ingreso_Mensual': [5000, 7000, 10000, 12000, 15000],
    'Genero': ['M', 'F', 'F', 'M', 'M'],
    'Fecha_Registro': ['2022-01-01', '2021-12-15', '2021-10-20', '2022-02-10', '2022-03-05'],
    'Gasto_Anual': [2000, 3000, 7000, 5000, 8000]
}
df = pd.DataFrame(data)

# Nombre
logger.info("Nombre: Gabriel Guzmán")
print("Nombre: Gabriel Guzmán\n")


# Imprimir y registrar el DataFrame inicial
logger.info("DataFrame inicial:")
print("DataFrame inicial:")
print(df)

# Conversión de la columna de fechas a formato datetime
df['Fecha_Registro'] = pd.to_datetime(df['Fecha_Registro'])
logger.info("Convertido 'Fecha_Registro' a formato datetime.")

# 1. Creación de nuevas variables: Días desde el registro
df['Dias_Desde_Registro'] = (pd.Timestamp.now() - df['Fecha_Registro']).dt.days
logger.info("Creada la columna 'Dias_Desde_Registro':\n%s", df[['Fecha_Registro', 'Dias_Desde_Registro']])

# 2. Codificación de variables categóricas: One-Hot Encoding para la columna de género
encoder = OneHotEncoder(sparse_output=False)
genero_encoded = encoder.fit_transform(df[['Genero']])
df_encoded = pd.DataFrame(genero_encoded, columns=encoder.get_feature_names_out(['Genero']))
df = pd.concat([df, df_encoded], axis=1).drop(columns=['Genero'])
logger.info("Codificación One-Hot realizada para la columna 'Genero':\n%s", df[df_encoded.columns])

# 3. Normalización y Estandarización
scaler_minmax = MinMaxScaler()
scaler_standard = StandardScaler()

df['Ingreso_Normalizado'] = scaler_minmax.fit_transform(df[['Ingreso_Mensual']])
df['Edad_Estandarizada'] = scaler_standard.fit_transform(df[['Edad']])
logger.info("Normalización y estandarización realizadas:\n%s", df[['Ingreso_Mensual', 'Ingreso_Normalizado', 'Edad', 'Edad_Estandarizada']])

# 4. Extracción de características de fecha
df['Mes_Registro'] = df['Fecha_Registro'].dt.month
df['Anio_Registro'] = df['Fecha_Registro'].dt.year
logger.info("Características de fecha extraídas: 'Mes_Registro' y 'Anio_Registro'.\n%s", df[['Fecha_Registro', 'Mes_Registro', 'Anio_Registro']])

# Análisis de correlación
logger.info("Análisis de correlación con 'Gasto_Anual' como variable objetivo.")
correlation_matrix = df.corr()
correlations_with_target = correlation_matrix['Gasto_Anual'].drop('Gasto_Anual')
logger.info("Correlaciones con 'Gasto_Anual':\n%s", correlations_with_target)

# Filtrar atributos con baja correlación
threshold = 0.25
relevant_features = correlations_with_target[abs(correlations_with_target) >= threshold].index.tolist()
df_reduced = df[relevant_features + ['Gasto_Anual']]  # Mantener solo los atributos relevantes y el objetivo

logger.info("Atributos relevantes seleccionados (correlación >= 0.1): %s", relevant_features)
print("\nDataFrame final con atributos relevantes:")
print(df_reduced)


INFO - Nombre: Gabriel Guzmán
INFO - DataFrame inicial:
INFO - Convertido 'Fecha_Registro' a formato datetime.
INFO - Creada la columna 'Dias_Desde_Registro':
  Fecha_Registro  Dias_Desde_Registro
0     2022-01-01                 1035
1     2021-12-15                 1052
2     2021-10-20                 1108
3     2022-02-10                  995
4     2022-03-05                  972
INFO - Codificación One-Hot realizada para la columna 'Genero':
   Genero_F  Genero_M
0       0.0       1.0
1       1.0       0.0
2       1.0       0.0
3       0.0       1.0
4       0.0       1.0
INFO - Normalización y estandarización realizadas:
   Ingreso_Mensual  Ingreso_Normalizado  Edad  Edad_Estandarizada
0             5000                  0.0    25           -1.293993
1             7000                  0.2    30           -0.754829
2            10000                  0.5    45            0.862662
3            12000                  0.7    35           -0.215666
4            15000                  

Nombre: Gabriel Guzmán

DataFrame inicial:
   Edad  Ingreso_Mensual Genero Fecha_Registro  Gasto_Anual
0    25             5000      M     2022-01-01         2000
1    30             7000      F     2021-12-15         3000
2    45            10000      F     2021-10-20         7000
3    35            12000      M     2022-02-10         5000
4    50            15000      M     2022-03-05         8000

DataFrame final con atributos relevantes:
   Edad  Ingreso_Mensual  Ingreso_Normalizado  Edad_Estandarizada  Gasto_Anual
0    25             5000                  0.0           -1.293993         2000
1    30             7000                  0.2           -0.754829         3000
2    45            10000                  0.5            0.862662         7000
3    35            12000                  0.7           -0.215666         5000
4    50            15000                  1.0            1.401826         8000
